This notebook is for exploring the use of multivaraible stepwise regression for gathering further evidence that 3D strain is important predictor of AF

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

from hrs_23_figures import *

from sklearn import metrics

regions=['global', 'roof', 'sept', 'lat', 'ant', 'post']

In [2]:
from sklearn.preprocessing import StandardScaler

In [3]:
path="/home/csi20/Dropbox/phd/Data/kiru_demographics_df_omitAFimaging.csv"

In [4]:
cases_df = pd.read_csv(path)

In [5]:
cases_df.columns

Index(['Case', 'Age', 'Sex', 'NYHA class', 'Device present?', 'AF',
       'pre ECG rhythm', 'Pacing changes for CT', 'Pacing',
       'Most recent pacing check prior to CT', 'CT heart rhythm', 'MS QRSd',
       'AL QRSd', 'Comments', 'CT date', 'Heart rate', 'LA strain',
       'LA volume', 'LA size', 'LA emptying fraction', 'LA pressure',
       'LV ES volume', 'LVES diameter (cm)', 'LVEF', 'MR/severity', 'HF meds',
       'Previous ablation', 'Prior mitral procedures', 'af_num', 'area_global',
       'area_roof', 'area_sept', 'area_lat', 'area_ant', 'area_post',
       'area_global_mshqual', 'area_roof_mshqual', 'area_sept_mshqual',
       'area_lat_mshqual', 'area_ant_mshqual', 'area_post_mshqual',
       'f1_global', 'f1_roof', 'f1_sept', 'f1_lat', 'f1_ant', 'f1_post',
       'f2_global', 'f2_roof', 'f2_sept', 'f2_lat', 'f2_ant', 'f2_post',
       'LA_vol_CT', 'LAEF_CT', 'Sex_num', 'LV_vol_CT', 'LVEF_CT',
       'area_global_vxm', 'area_roof_vxm', 'area_sept_vxm', 'area_lat_vxm',


In [114]:
# list_of_input_vars = ['LA_vol_CT', 'LAEF_CT', 'area_global', 'f1_global',
#                       'area_roof', 'f1_roof', 'area_sept', 'f1_sept', 'area_lat', 'f1_lat',
#                       'area_ant', 'f1_ant', 'area_post', 'f1_post']

base_input_vars = ['Age', 'LA_vol_CT']

variables = ['LAEF_CT', 'f1_global', 'f1_roof', 'f1_sept', 'f1_lat', 'f1_ant', 'f1_post']
# variables = ['area_global', 'area_roof', 'area_sept', 'area_lat', 'area_ant', 'area_post']

for var_to_add in variables:
    
    list_of_input_vars = base_input_vars + [var_to_add]

    # Assuming your data is in a DataFrame called 'data'
    X = cases_df[list_of_input_vars]
    y = cases_df['af_num']

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    X_scaled
    X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)


    included = []
    best_model = None

    while True:
        # Variables not yet included in the model
        remaining = list(set(X_scaled_df.columns) - set(included))

        if len(remaining) == 0:
            break

        # Store models and their AIC scores
        models = []

        for feature in remaining:
            model = sm.Logit(y, sm.add_constant(X_scaled_df[included + [feature]])).fit()
            models.append((model, model.aic))

        # Choose the model with the lowest AIC score
        best_model, best_aic = min(models, key=lambda x: x[1])

        # Terminate the loop if the current model is worse than the previous best model
        if best_model.aic > best_aic:
            break

        # Update the included variables
        included.append(remaining[models.index((best_model, best_aic))])


    final_model = sm.Logit(y, sm.add_constant(X[included])).fit()
    print(f"########################### {var_to_add} #####################################")
    print("\n", final_model.summary(), "\n")


Optimization terminated successfully.
         Current function value: 0.347977
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.593487
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.385251
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.344078
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.324955
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.318128
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.318128
         Iterations 8
########################### LAEF_CT #####################################

                            Logit Regression Results                           
Dep. Variable:                 af_num   No. Observations:                   30
Model:                          Logit   Df Residuals

Optimization terminated successfully.
         Current function value: 0.353623
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.293488
         Iterations 9
Optimization terminated successfully.
         Current function value: 0.293205
         Iterations 9
Optimization terminated successfully.
         Current function value: 0.293205
         Iterations 9
########################### f1_post #####################################

                            Logit Regression Results                           
Dep. Variable:                 af_num   No. Observations:                   30
Model:                          Logit   Df Residuals:                       26
Method:                           MLE   Df Model:                            3
Date:                Wed, 05 Jul 2023   Pseudo R-squ.:                  0.5200
Time:                        14:13:08   Log-Likelihood:                -8.7961
converged:                       True   LL

In [149]:
# list_of_input_vars = ['LA_vol_CT', 'LAEF_CT', 'area_global', 'f1_global',
#                       'area_roof', 'f1_roof', 'area_sept', 'f1_sept', 'area_lat', 'f1_lat',
#                       'area_ant', 'f1_ant', 'area_post', 'f1_post']

base_input_vars = ['Age', 'LA_vol_CT']

variables = ['LAEF_CT', 'f1_global', 'f1_roof', 'f1_sept', 'f1_lat', 'f1_ant', 'f1_post']
variables = ['LAEF_CT', 'area_global', 'area_roof', 'area_sept', 'area_lat', 'area_ant', 'area_post']

for var_to_add in variables:
    
    list_of_input_vars = base_input_vars + [var_to_add]

    # Assuming your data is in a DataFrame called 'data'
    X = cases_df[list_of_input_vars]
    y = cases_df['af_num']

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    model = sm.Logit(y, sm.add_constant(X_scaled)).fit()
    standardized_coefficients = model.params[1:]
    pvalues = model.pvalues[1:]
    
    # Print variable names and their coefficients
    print(f"\n ################ {var_to_add} ##############")
    for name, coef, pval in zip(list_of_input_vars, standardized_coefficients, pvalues):
        print(f"{name}: \t\t\t{np.round(coef,2)} \t\t{np.round(pval,2)}")

Optimization terminated successfully.
         Current function value: 0.318128
         Iterations 8

 ################ LAEF_CT ##############
Age: 			-0.38 		0.52
LA_vol_CT: 			1.05 		0.25
LAEF_CT: 			-2.25 		0.09
Optimization terminated successfully.
         Current function value: 0.239071
         Iterations 9

 ################ area_global ##############
Age: 			-0.31 		0.71
LA_vol_CT: 			1.6 		0.11
area_global: 			-5.61 		0.03
Optimization terminated successfully.
         Current function value: 0.330067
         Iterations 8

 ################ area_roof ##############
Age: 			-0.06 		0.92
LA_vol_CT: 			2.43 		0.02
area_roof: 			-1.32 		0.09
Optimization terminated successfully.
         Current function value: 0.283425
         Iterations 9

 ################ area_sept ##############
Age: 			-0.26 		0.7
LA_vol_CT: 			2.39 		0.03
area_sept: 			-4.21 		0.06
Optimization terminated successfully.
         Current function value: 0.308964
         Iterations 8

 ################ a

In [156]:
# Base and additional independent variables
base_input_vars = ['Age', 'LA_vol_CT', 'LAEF_CT']
# variables = ['LAEF_CT', 'f1_global', 'f1_roof', 'f1_sept', 'f1_lat', 'f1_ant', 'f1_post']
variables = ['area_global', 'area_roof', 'area_sept', 'area_lat', 'area_ant', 'area_post',
            'f1_global', 'f1_roof', 'f1_sept', 'f1_lat', 'f1_ant', 'f1_post']

# Create an empty DataFrame to store the results
results_df = pd.DataFrame(columns=['Variable', 'Coefficient', 'P-value'])

for var_to_add in variables:
    list_of_input_vars = base_input_vars + [var_to_add]

    # Assuming your data is in a DataFrame called 'cases_df'
    X = cases_df[list_of_input_vars]
    y = cases_df['af_num']

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    model = sm.Logit(y, sm.add_constant(X_scaled)).fit()
    standardized_coefficients = model.params[1:]
    pvalues = model.pvalues[1:]
    
    # Append results to the DataFrame
    for name, coef, pval in zip(list_of_input_vars, standardized_coefficients, pvalues):
        results_df = results_df.append({'Variable': name, 'Coefficient': np.round(coef,2), 'P-value': np.round(pval,2)}, ignore_index=True)

# Print the results table
print(results_df)


Optimization terminated successfully.
         Current function value: 0.225172
         Iterations 11
Optimization terminated successfully.
         Current function value: 0.286581
         Iterations 9
Optimization terminated successfully.
         Current function value: 0.279463
         Iterations 9
Optimization terminated successfully.
         Current function value: 0.286724
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.314262
         Iterations 9
Optimization terminated successfully.
         Current function value: 0.242526
         Iterations 9
Optimization terminated successfully.
         Current function value: 0.224137
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.242825
         Iterations 9
Optimization terminated successfully.
         Current function value: 0.260795
         Iterations 9
Optimization terminated successfully.
         Current function value: 0.250723


In [157]:
results_df.to_csv("/home/csi20/stepwise_regression_la_strain.csv")

In [145]:
# list_of_input_vars = ['LA_vol_CT', 'LAEF_CT', 'area_global', 'f1_global',
#                       'area_roof', 'f1_roof', 'area_sept', 'f1_sept', 'area_lat', 'f1_lat',
#                       'area_ant', 'f1_ant', 'area_post', 'f1_post']

# base_input_vars = ['Age', 'LA_vol_CT', 'LAEF_CT']
base_input_vars = ['LA_vol_CT', 'f1_global', 'f1_sept', 'f1_lat', 'f1_ant', 'f1_post']

# Assuming your data is in a DataFrame called 'data'
X = cases_df[base_input_vars]
y = cases_df['af_num']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

model = sm.Logit(y, sm.add_constant(X_scaled)).fit(maxiter=200)
standardized_coefficients = model.params[1:]
pvalues = model.pvalues[1:]

# Print variable names and their coefficients
print("\n")
for name, coef, pval in zip(base_input_vars, standardized_coefficients, pvalues):
    print(f"{name}: \t\t\t{coef} \t\t{pval}")

         Current function value: inf
         Iterations: 200


/home/csi20/anaconda3/envs/vtk/lib/python3.9/site-packages/statsmodels/discrete/discrete_model.py:2383: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/home/csi20/anaconda3/envs/vtk/lib/python3.9/site-packages/statsmodels/discrete/discrete_model.py:2441: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q * linpred)))


LinAlgError: Singular matrix

In [125]:
model.pvalues

const    0.057946
x1       0.895817
x2       0.105834
x3       0.102184
dtype: float64